In [18]:
import networkx as nx
import json

def load_multigraph(graph_json_path):
    """
    Loads the saved MultiGraph from a JSON file.
    
    :param graph_json_path: Path to the saved graph JSON file.
    :return: NetworkX MultiGraph object.
    """
    with open(graph_json_path, "r") as f:
        data = json.load(f)
    return nx.node_link_graph(data)

def query_polypharmacy_risk(graph_json_path, drug_list, k):
    """
    Queries the graph to find the top-k highest-risk polypharmacy interactions for each drug pair.
    
    :param graph_json_path: Path to the saved graph JSON file.
    :param drug_list: List of drugs input by the user.
    :param k: Number of top interactions to return for each drug pair.
    :return: Dictionary with drug pairs as keys and top-k interactions as values.
    """
    # Load the graph
    G = load_multigraph(graph_json_path)

    results = {}

    # Find interactions within the user's drug list
    for d1 in drug_list:
        for d2 in drug_list:
            if d1 < d2 and G.has_edge(d1, d2):  # Ensure unique order for pairs
                pair_results = []

                # Iterate through all hyper-relations between (d1, d2)
                for _, edge_data in G.get_edge_data(d1, d2).items():
                    pair_results.append((d1, d2, edge_data["adverseEvent"], edge_data["PRR"]))

                # Sort by PRR in descending order and take the top k
                pair_results = sorted(pair_results, key=lambda x: x[3], reverse=True)[:k]

                if pair_results:
                    results[(d1, d2)] = pair_results

    return results



In [21]:
if __name__ == "__main__":
    graph_json_path = "../output/test/graph/polypharmacy_multigraph.json"  
    user_drugs = ["Temazepam", "sildenafil", "Prednisone", "Cyclophosphamide", "zopiclone"] # List of drugs
    top_k = 5  

    risks = query_polypharmacy_risk(graph_json_path, user_drugs, top_k)

    print("\nTop Polypharmacy Risks:")
    for (drug1, drug2), interactions in risks.items():
        print(f"\n{drug1} + {drug2}:")
        for idx, (_, _, condition, prr) in enumerate(interactions, 1):
            print(f"  {idx}. {drug1} + {drug2} → {condition} (PRR: {prr})")


Top Polypharmacy Risks:

Temazepam + sildenafil:
  1. Temazepam + sildenafil → Economic problem (PRR: 50.0)
  2. Temazepam + sildenafil → Obesity (PRR: 40.0)
  3. Temazepam + sildenafil → Hypoxia (PRR: 26.6667)
  4. Temazepam + sildenafil → Haematochezia (PRR: 20.0)
  5. Temazepam + sildenafil → Osteonecrosis of jaw (PRR: 20.0)

Cyclophosphamide + zopiclone:
  1. Cyclophosphamide + zopiclone → Neutropenic sepsis (PRR: 50.0)
  2. Cyclophosphamide + zopiclone → Toxic epidermal necrolysis (PRR: 40.0)
  3. Cyclophosphamide + zopiclone → Metastases to bone (PRR: 40.0)
  4. Cyclophosphamide + zopiclone → Febrile neutropenia (PRR: 30.0)
  5. Cyclophosphamide + zopiclone → Colitis ischaemic (PRR: 30.0)
